Skip to section **Build Endpoint** if you already built Docker image and pushed it to ECR 

In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:20.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

ARG PYTHON_VERSION_TAG=3.8.3
ARG LINK_PYTHON_TO_PYTHON3=1

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*
    
RUN apt-get -qq -y update && \
    DEBIAN_FRONTEND=noninteractive apt-get -qq -y install \
        gcc \
        g++ \
        zlibc \
        zlib1g-dev \
        libssl-dev \
        libbz2-dev \
        libsqlite3-dev \
        libncurses5-dev \
        libgdbm-dev \
        libgdbm-compat-dev \
        liblzma-dev \
        libreadline-dev \
        uuid-dev \
        libffi-dev \
        tk-dev \
        curl \
        git \
        make \
        sudo \
        bash-completion \
        tree \
        vim \
       

# Docker Image

Befor building the image, you need to make the container able to run the file ./container/ProtCNN/serve. To do so, run the follwiong command in the terminal: <br>
<br>
chmod +x container/ProtCNN/serve <br>
<br>
Then, you can proceed with building the image.

In [1]:
%cd container
!docker build -t port-cnn .
%cd ../

/home/ec2-user/SageMaker/protein-annotation/inference-container/container
Sending build context to Docker daemon  377.9kB
Step 1/17 : FROM ubuntu:20.04
 ---> f643c72bc252
Step 2/17 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 326b3d86eee1
Step 3/17 : ARG PYTHON_VERSION_TAG=3.8.3
 ---> Using cache
 ---> 439310e2c915
Step 4/17 : ARG LINK_PYTHON_TO_PYTHON3=1
 ---> Using cache
 ---> dbb5d06e3d11
Step 5/17 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5f43ad46e6dd
Step 6/17 : RUN apt-get -qq -y update &&     DEBIAN_FRONTEND=noninteractive apt-get -qq -y install         gcc         g++         zlibc         zlib1g-dev         libssl-dev         libbz2-dev         libsqlite3-dev         libncurses5-dev         libgdbm-dev         libgdbm-compat-dev         liblzma-dev         libreadline-dev         uuid-dev         libffi-dev    

In [ ]:
# Push image to ECR
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'port-cnn'
tag = ':latest'
uri_suffix = 'amazonaws.com'
port_cnn_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $port_cnn_uri
!docker push $port_cnn_uri

# Build Endpoint

## Initializations

In [1]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [3]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/port-cnn:latest'.format(account, region)

In [4]:
sage = boto3.Session().client(service_name='sagemaker')

## Create Model

In [5]:
model_name = "PortCNN-prediction"

In [6]:
# Add the location of model artifacts to ModelDataUrl field
primary_container = {
    'Image': image,
    'ModelDataUrl': "",
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

## Create Endpoint Configuration

In [7]:
job_name_prefix = 'PortCNN-inference'

In [ ]:
import time
from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

## Create Endpoint

In [9]:
sagemaker = boto3.client(service_name='sagemaker')

In [ ]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

In [11]:
# get the status of the endpoint
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sagemaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

EndpointStatus = Creating
Endpoint creation ended with EndpointStatus = InService


## Invoke Endpoint

In [27]:
import boto3
import re
import csv
import json

In [22]:
# Read sample test
sample_test = []

with open('sample-test.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        sample_test.append(row[0])

In [23]:
sample_test

['RHARLVSRFSDLYTVERLDAQTLLRRYIPDIEMVQRIIFIAVMESFQKAKLAYRKFKQQVRKTLSTSHFGPESLEDAAVDYIVRNLDLYDVLCSVNVRENDFVFSRKVFFQPVTTFCMYVLPPYLSALIKKNPATSSCSPPLLYIAPTSRGHSNIMYRRSFDSDFSAPLVVYYVWPALVEGSTVLVKGEA',
 'LDVEIADTDPKREQGLMFRRSLSENQGMIFLFGREREITMWMKNTFIPLDMVFIGDDWRVVSIAQNAEPFSTDVISSRRPASRVLEIGAGQAKKLGLKVGDRVSL',
 'EKLEVWKLSKNFATKIYKNTENFPNEEKFGLVSQLRRAAVSVASNLAEGSSRKSKKDQAHFSQIAYSSLMEVLCQLEIAKDIGYISENDLQDLRSDASKIAYMINS',
 'SLSEARRFNTSYVGTEHILLGLLREGEGVAVRILMEQGIDFNRVREEIVKMLS',
 'AINELKKELKAVILAHYYQDPDIQDIADYIGDSLGLSQQAATTDKEVIVFAGVHFMAETAKILNPDKLVLLPDLEAGCSLADSCPPEEFAQFKTQYPDAIVVSYINCTADIKAMSDVICTSSNAVKIVNQLPKDRPIIFGPDRNLGRYVAQQTGRDLILWQGSCIVHETFSERRIVQLKIEHPSAEIIAHPECEEPVLRHANYIGSTTALLKYSQQSPQDSFIVATEPGIIHQMQKEAPNKTFIPAPAMNNCACNECPYMRLNTLEKLYLAMKHKQPEIIMDESTRKAALKPIQRMLE']

In [20]:
runtime = boto3.Session().client(service_name='runtime.sagemaker')

In [24]:
# Invoke model with a single sample
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType= 'text/csv', 
                                           Body=sample_test[0] + "\r\n")
print(response)

{'ResponseMetadata': {'RequestId': 'a036c535-ec7d-45be-a378-50f7cdf239c1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a036c535-ec7d-45be-a378-50f7cdf239c1', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Tue, 5 Jan 2021 20:23:20 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '37'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7fbbd34b26a0>}


In [25]:
# response needs some processing to remove the five characters ' AND " AND ( AND ) AND \n
r = response['Body'].read().decode()
print(r)

"('PF16026.5', 'MIEAP', 0.99273646)"



In [26]:
# Match ' OR " OR ( OR ) OR \n
# The result is a list containing Family Accession, Family ID, and confidence score
re.sub("\'|\"|\(|\)|\n", '', r).split(", ")

['PF16026.5', 'MIEAP', '0.99273646']

In [28]:
responses = []
for payload in sample_test:
    if (payload != ''):
        response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType= 'text/csv', 
                                           Body=payload + "\r\n")
        response = json.loads(response['Body'].read().decode())
        response = re.sub("\'|\"|\(|\)|\n", '', response).split(", ")
        response.append(payload)
        
        responses.append(tuple(response))

In [29]:
responses

[('PF16026.5',
  'MIEAP',
  '0.99273646',
  'RHARLVSRFSDLYTVERLDAQTLLRRYIPDIEMVQRIIFIAVMESFQKAKLAYRKFKQQVRKTLSTSHFGPESLEDAAVDYIVRNLDLYDVLCSVNVRENDFVFSRKVFFQPVTTFCMYVLPPYLSALIKKNPATSSCSPPLLYIAPTSRGHSNIMYRRSFDSDFSAPLVVYYVWPALVEGSTVLVKGEA'),
 ('PF02643.15',
  'DUF192',
  '0.9993938',
  'LDVEIADTDPKREQGLMFRRSLSENQGMIFLFGREREITMWMKNTFIPLDMVFIGDDWRVVSIAQNAEPFSTDVISSRRPASRVLEIGAGQAKKLGLKVGDRVSL'),
 ('PF05635.11',
  '23S_rRNA_IVP',
  '0.99970835',
  'EKLEVWKLSKNFATKIYKNTENFPNEEKFGLVSQLRRAAVSVASNLAEGSSRKSKKDQAHFSQIAYSSLMEVLCQLEIAKDIGYISENDLQDLRSDASKIAYMINS'),
 ('PF02861.20',
  'Clp_N',
  '0.8881429',
  'SLSEARRFNTSYVGTEHILLGLLREGEGVAVRILMEQGIDFNRVREEIVKMLS'),
 ('PF02445.16',
  'NadA',
  '0.99999774',
  'AINELKKELKAVILAHYYQDPDIQDIADYIGDSLGLSQQAATTDKEVIVFAGVHFMAETAKILNPDKLVLLPDLEAGCSLADSCPPEEFAQFKTQYPDAIVVSYINCTADIKAMSDVICTSSNAVKIVNQLPKDRPIIFGPDRNLGRYVAQQTGRDLILWQGSCIVHETFSERRIVQLKIEHPSAEIIAHPECEEPVLRHANYIGSTTALLKYSQQSPQDSFIVATEPGIIHQMQKEAPNKTFIPAPAMNNCACNECPYMRLNTLEKLYLAMKHKQPEIIMDESTRKAALKPIQRM